# DARPA-ARC Notebook 2: fMRI Postprocessing

## Compute Motion Summary Statistics of and Visualize Motion

In [ ]:
import os
import numpy as np
import pylab as plt
from collections import defaultdict
from pandas import DataFrame, read_csv
def demean(arr): return arr - arr.mean()

mri_dir = '/autofs/space/lilli_002/users/DARPA-ARC'
behavior_dir = '/space/sophia/2/users/DARPA-Behavior/arc/csv'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

thresholds = [0.5,0.9,1.3]
selected_threshold = 0.9
tr = 1.75

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

info = read_csv('demographics.csv')
subjects = info.loc[~info.Exlude,'Subject'].as_matrix()
stats = defaultdict(list)

for subject in subjects:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare MRI data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load gray/white matter timeseries.
    npz = np.load('fmri_motion/%s_arc_qc_data.npz' %subject)
    gm = np.apply_along_axis(demean, 1, npz['gm'])
    wm = np.apply_along_axis(demean, 1, npz['wm'])
    
    ''' A NOTE ON MOTION DATA.
    1) Understanding Freesurfer motion outputs: https://mail.nmr.mgh.harvard.edu/pipermail//freesurfer/2013-May/030273.html
    2) Understanding angular displacement: https://en.wikipedia.org/wiki/Angular_displacement
    3) Understanding framewise displacement: See Power 2012, 2014
    '''

    ## Read motion data.
    mc = os.path.join(mri_dir, subject, 'arc_001', '001', 'fmcpr.mcdat')
    mc = np.loadtxt(mc)[:,1:7]

    ## Invert angular displacement.
    mc[:,:3] = np.deg2rad(mc[:,:3]) # Convert degrees to radians
    mc[:,:3] *= 50                  # Convert radians to mm [Following Power 2012, we assume a head ~ sphere w/ r=50mm]

    ## Compute framewise displacement (See Power 2012, 2014).
    fd = np.insert( np.abs( np.diff(mc, axis=0) ).sum(axis=1), 0, 0 )

    ## Compute absolute displacement. 
    rot = ( np.abs( mc - mc[0] )[:,:3] ).sum(axis=1)
    trans = ( np.abs( mc - mc[0] )[:,3:] ).sum(axis=1)
    
    ## Compute volumes to remove.
    rejections = np.zeros_like(thresholds)
    for n, threshold in enumerate(thresholds): rejections[n] += (fd >= threshold).sum()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare behavior data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Load behavior CSV.
    df = read_csv(os.path.join(behavior_dir, '%s_arc_mri_ser-1' %subject))
    
    ## Create onsets of each TR.
    onsets = np.cumsum( np.ones_like(fd) * tr )
    onsets = np.insert(onsets, 0, 0)

    ## Define onsets/offsets of trials & shocks.
    trial_starts = df.RiskOnset.as_matrix()
    trial_ends = np.append(df.FixOnset[1:], trial_starts[-1] + 5.25)
    shock_starts = df.ShockOnset[~df.ShockOnset.isnull()].as_matrix()
    shock_ends = np.array([trial_ends[np.argmin((trial_ends - s)**2)] for s in shock_starts])

    ## Digitize onsets/offsets.
    trial_starts = np.digitize(trial_starts, onsets)
    trial_ends = np.digitize(trial_ends, onsets)
    shock_starts = np.digitize(shock_starts, onsets)
    shock_ends = np.digitize(shock_ends, onsets)
    
    ## Make boxcars for plotting.
    trials = np.zeros_like(fd)
    for i,j in zip(trial_starts,trial_ends): trials[i:j+1] += 1 
        
    shocks = np.zeros_like(fd)
    for i,j in zip(shock_starts,shock_ends): shocks[i:j+1] += 1 

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Calculate summary statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats['Subject'] += [subject]
    
    ## Add motion information.
    stats['Abs_Disp_Rot']   += [rot.max()]
    stats['Abs_Disp_Trans'] += [trans.max()]
    stats['FD_mean'] += [fd.mean()]
    stats['FD_sd']   += [fd.std()]
    stats['FD_max']  += [fd.max()]
    
    ## Calculate number of rejections.
    fd_index, = np.where(fd >= selected_threshold)
    n_reject = len(fd_index)
    stats['FD_reject'] += [n_reject]
    
    ## Calculate fraction of rejected displacements across all 
    ## instances of a portion of the run.
    stats['FD_frac_task']  += [((trials) * (fd >= selected_threshold)).mean()]
    stats['FD_frac_rest']  += [((1 - trials) * (fd >= selected_threshold)).mean()]
    stats['FD_frac_shock'] += [((shocks) * (fd >= selected_threshold)).mean()]
    
    ## Calculate percentages of rejection displacesments
    ## within given categories (non-shock task, shock, rest)
    stats['FD_perc_rest']    += [(1-trials)[fd_index].sum() / float(n_reject) ]
    stats['FD_perc_shock']   += [shocks[fd_index].sum() / float(n_reject) ]
    stats['FD_perc_task_ns'] += [1 - stats['FD_perc_rest'][-1] - stats['FD_perc_shock'][-1]]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save summary statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
stats = DataFrame(stats)
stats = stats[['Subject', 'Abs_Disp_Rot', 'Abs_Disp_Trans', 'FD_mean', 'FD_sd', 'FD_max',
               'FD_reject', 'FD_frac_task', 'FD_frac_shock', 'FD_frac_rest',
               'FD_perc_task_ns',  'FD_perc_shock', 'FD_perc_rest']]
stats.to_csv('fmri_motion/motion_stats.csv', index=False)

print 'Done.'